# Creating a Stigma Classification Model for Tweets

##### Sagar Jilka (sagar.jilka@kcl.ac.uk)

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from textblob.classifiers import NaiveBayesClassifier
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns
nltk.download('stopwords')
%matplotlib inline
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
pd.set_option('display.max_columns', 500)
##Explore this blog: https://jakevdp.github.io/PythonDataScienceHandbook/05.08-random-forests.html for visualisation ideas

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ertur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Import Data 

In [2]:
fn = 'ALL_Schizophrenia_Tweets.csv'

In [3]:
data = pd.read_csv(fn, header=None, encoding = 'Latin1', names = ['body_text', 'label', 'wat', 'wat2'], skiprows=1)#, sep='\t')

In [4]:
data = data[['label', 'body_text']]

In [5]:
data.shape

(1000, 2)

In [6]:
data.head()

,label,body_text
0,0,Was the Police Shooting of Psychotic Woman Jus...
1,1,"@WikiGuido In fairness, in our psychotic count..."
2,0,Sorry. My imagination moves a lot faster than...
3,0,@shnerdy_girdy @T_Ivan17 @Victoriaj1004 So my ...
4,1,Cop Trial Begins in Shooting Death of Schizoph...


# Pre-process Data 

In [7]:
print("the number of rated tweets by category:")

data.groupby('label')['body_text'].nunique()

the number of rated tweets by category:


label
0    446
1    155
2    299
o      1
Name: body_text, dtype: int64

In [8]:
data.label.replace('o' , "0", inplace=True)

In [9]:
print("input data has {} row and {} columns".format(len(data), len(data.columns)))

input data has 1000 row and 2 columns


In [10]:
print("out of {} rows,\n\n {} are 2,\n\n {} are 1,\n\n {} are 0,\n\n  {} have no rating and will be dropped".format(len(data),
                                                               len(data[data['label'] == "2"]),
                                                               len(data[data['label'] == "1"]),
                                                               len(data[data['label'] == "0"]),
                                                               data['label'].isnull().sum()))

out of 1000 rows,

 299 are 2,

 155 are 1,

 447 are 0,

  99 have no rating and will be dropped


In [11]:
## Now we are making this a binary classification problem by only keeping the 0s and 2s.
# We will also drop the tweets with missing labels
# 0 = Not stigmatizing
# 1 = Stigmatizing

data = data[data.label != '1']
data['label'] = data.label.replace('2', '1')
data = data.dropna()

In [12]:
print("the data after pre-processing and cleaning, which includes converting the classifications into a binary model, looks like:")

the data after pre-processing and cleaning, which includes converting the classifications into a binary model, looks like:


In [13]:
data

,label,body_text
0,0,Was the Police Shooting of Psychotic Woman Jus...
2,0,Sorry. My imagination moves a lot faster than...
3,0,@shnerdy_girdy @T_Ivan17 @Victoriaj1004 So my ...
5,1,"Devon, the more I watch you in Nikita . The mo..."
7,1,Tonight's #StateOfTheUnion you can expect Dona...
...,...,...
991,1,@UmmAlMumineen I don't know between that crazy...
992,0,Apparently BPD = soc*op*thy now?\n\nLet's igno...
993,1,thomas as a psychotic serial killer being hunt...
996,0,@Trumpfan1995 https://t.co/7KOdgkgYBL


In [14]:
print("out of {} rows,\n\n {} are 2,\n\n {} are 1,\n\n {} are 0,\n\n  {} have no rating and will be dropped".format(len(data),
                                                               len(data[data['label'] == "2"]),
                                                               len(data[data['label'] == "1"]),
                                                               len(data[data['label'] == "0"]),
                                                               data['label'].isnull().sum()))

out of 746 rows,

 0 are 2,

 299 are 1,

 447 are 0,

  0 have no rating and will be dropped


In [15]:
data.label = data.label.astype(int)

In [16]:
(data.label == 0).sum() / len(data)

0.599195710455764

In [17]:
(data.label == 1).sum() / len(data)

0.4008042895442359

## Feature Extraction

In [18]:
# Calculate sentiment and subjectivity for each tweet. We will use these values as features in our model.
# i.e. are tweets that are classed as stigmatizing more negative?
def sentAnal(df):
    for index, row in df.iterrows():
        temp = TextBlob(row['body_text'])
        df.loc[index,'Sentiment'] = temp.sentiment.polarity
        df.loc[index,'Subjectivity'] = temp.sentiment.subjectivity
    return df

In [19]:
data = sentAnal(data)

In [20]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100


data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [22]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

In [23]:
# Average Word Length. simply take the sum of the length of all the words and divide it by the total length of the tweet as defined in function above
data['avg_word'] = data['body_text'].apply(lambda x: avg_word(x))

In [24]:
# Number of Words in tweet
data['word_count'] = data['body_text'].apply(lambda x: len(str(x).split(" ")))

In [25]:
# number of numerics in tweet
data['numerics'] = data['body_text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

In [26]:
## remove rare words?
freq = pd.Series(' '.join(data['body_text']).split()).value_counts()[-10:]

freq = list(freq.index)
data['body_text'] = data['body_text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [27]:
freq

['@ElizabethMaho11',
 'unqualified',
 'extremists',
 'scared',
 'Patients',
 'intention',
 'prodromal',
 'drug.',
 '@SallyQYates',
 'PSYCHIATRY,']

In [28]:
data.head()

,label,body_text,Sentiment,Subjectivity,punct%,avg_word,word_count,numerics
0,0,Was the Police Shooting of Psychotic Woman Jus...,-0.050000,0.950000,9.9,6.250000,16,0
2,0,Sorry. My imagination moves a lot faster than ...,-0.250000,0.500000,2.5,5.000000,17,0
3,0,@shnerdy_girdy @T_Ivan17 @Victoriaj1004 So my ...,-0.714286,0.857143,5.3,5.896552,29,0
5,1,"Devon, the more I watch you in Nikita . The mo...",0.166667,0.666667,5.5,3.560976,41,0
7,1,Tonight's #StateOfTheUnion you can expect Dona...,0.247619,0.439286,2.6,4.872340,47,0


In [29]:
from scipy import stats

In [30]:
cols = data[data.columns.difference(["label", "body_text"])].columns

In [31]:
cols

Index(['Sentiment', 'Subjectivity', 'avg_word', 'numerics', 'punct%',
       'word_count'],
      dtype='object')

## Split Data into training/testing sets 

In [32]:
#  making cols variable so that we have all feature columns names except our label
cols = data[data.columns.difference(["label"])].columns

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[cols], data['label'], test_size=0.2, random_state = 1)

print("The size of each training and testing datasets are:")
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

The size of each training and testing datasets are:
(596, 7)
(150, 7)
(596,)
(150,)


## Vectorize Text for hold-out test
#### Using inverse document frequency weighting (TF_IDF). This vectorizing method still creates a document term matrix (i.e. one row per tweet and columns representing single unique words), but with tfidf, instead of the cells representing the count, they represent a weighting that's meant to identify how important a word is to an individual tweet. The rarer the word is, the higher the tfifd value will be. This method helps to pull out important (but seldom used) words.

In [34]:
cols = data[data.columns.difference(["label", "body_text"])].columns

In [35]:
# instantiate the vectorizer
tfidf_vect = TfidfVectorizer(analyzer=clean_text)

# learn training data vocabulary, then use it to create a document-term matrix
tfidf_vect_fit = tfidf_vect.fit(X_train['body_text'])
tfidf_train = tfidf_vect_fit.transform(X_train['body_text'])


# transform testing data (using fitted vocabulary) into a document-term matrix
tfidf_test = tfidf_vect_fit.transform(X_test['body_text'])

X_train_vect = pd.concat([X_train[cols].reset_index(drop=True), 
           pd.DataFrame(tfidf_train.toarray(), columns=tfidf_vect.get_feature_names())], axis=1)

X_test_vect = pd.concat([X_test[cols].reset_index(drop=True), 
           pd.DataFrame(tfidf_test.toarray(), columns=tfidf_vect.get_feature_names())], axis=1)

print(X_train_vect.shape)
print("Our vectorized data looks like:")
X_train_vect

(596, 3836)
Our vectorized data looks like:


,Sentiment,Subjectivity,avg_word,numerics,punct%,word_count,,0,041,1,10,100,1000,100000,12,1200pm,12th,14th,14yr,15,17yearold,1979,1jasonbuchholz,1k,1st,1u,2,20,2003,2016,2017,2018,20yearold,22americanfirst,24,25,27,2xlp,3,30,30k,4,40calbulleit,448,448psychosi,4way,5,50,518,52kiii,53,537,6,60,68yr,6yearold,7,71,720p,7854479161,7pm,8,80,86,89,9,90,911,915fm,99,aaaaafffff,aavak,ab,abandon,abcnew,abduct,aberr,abil,abl,ableism,abolish,abondon,abort,abram,absolut,absurd,abt,abus,academ,accept,access,accesss,accid,accident,accompani,accord,account,accur,accus,achiev,acknowledg,acquiesc,across,act,action,activ,actorssing,actual,acut,ad,adapt,adavenh,add,addadhd,addict,addit,addon,address,adhd,adjust,adolesc,adult,advanc,advertis,affair,affect,afraid,african,afrin,agap,age,agenda,agent,aggrand,aggressor,agit,ago,agohaven,agre,ahead,aid,aileen,aim,airport,aka,al,album,alcohol,alex,aliv,allah,allegedli,allenfrancesmd,allevi,alley,allmus,allow,alon,along,along3,alongsid,alot,alreadi,also,altern,although,alway,alyssamilano,alzheim,alzheimersdiseas,amatur,amaz,america,american,americanhotlip,ami,among,amp,amphetamin,amput,amvetsupport,amwrit,analysi,anarchist,anc,andr,andrew,ang,angel,angri,anim,annual,anoth,answer,anti,antidepress,antidepressanthttpstco3smhc0hryv,antipsychot,antisemit,antisoci,anxieti,anymor,anyon,anyth,anyway,anywaya,anywher,ap,apa,apollo,appar,appbipolardisord,appear,appli,appointe,approach,appropri,april,aqa,aqapsycholog,arab,arbit,area,arent,arizona,arizonakayt,armi,arnold,aroesti,around,arriv,arsen,art,artefr,arthriti,articl,asf,asham,ashkenazi,ashwini,ashwininangia,asia,ask,aspd,aspir,ass,assassin,assault,assess,asshol,...,tutor,tv,tvbandnew,tweet,twilightavian,twist,twitter,two,typic,typpex,typpexprog,tyrant,u,u2,ualberta,ubc,uk,ukchang,um,umehoma,umma,ummah,ummalmumineen,un,unbridl,uncontrol,understand,underukslaw,undiagnos,unfair,unfold,unfollowblock,uniform,uninform,union,uniqu,unit,unitedfronthk,univers,unjustifi,unlac,unless,unlik,unmed,unpick,unquest,unreal,unrel,unsaf,unse,unstabl,untreat,updat,upload,upon,upscal,upthegrov,ur,urbanex,urgent,urself,us,usa,usatoday,use,usedup,useless,user,usual,utopia,utter,utterli,vain,valid,valiumtardivedyskinesia,valu,vancouv,variant,variat,variou,vast,vehicl,vengaadash,venom,vent,verbal,versail,version,via,vibe,vickyjane64,victim,victimolog,victoriaj1004,vid,video,view,villag,villain,villamarshmello,vincegottalotta,vinni,vinyl,violenc,violent,violet,viral,virgo,viru,visit,vital,voguemagazin,voic,volatil,volum,vote,vs,vu,w,wa,wafer,waffl,wage,wait,walk,walter,want,war,ward,warlord,warrenelli,washington,wasnt,wasp,watch,water,wave,way,wbuckley59,wcwinc,weak,weapon,weaselzipp,webinar,webmd,websit,wed,wednesdaywisdom,weed,week,weekend,weight,weird,weirdo,welcom,well,welldon,wellmindmd,welsh,went,weprogress,west,western,westminst,westsidejack37,what,whatev,whe,whenev,wherea,whether,whip,white,whiter,whole,whom,whore,wht,wide,widespread,wife,will,williamhaguemp,win,wing,winnipeg,wipe,wire,withdraw,withdrawn,within,without,woman,women,wonder,wood,woooooooo,word,work,world,worri,wors,worship,worst,worth,would,wouldnt,wow,wrap,write,writer,written,wrong,wrt,wterrorist,wtf,wth,wwwreserveinfus,xavier,xd,xi,xijinp,xpress,xxrealist,yall,yashar,yeah,year,yearli,yess,yesterday,yet,yg,yo,yolken,yondu,york,young,your,youtub,youv,yr,yu,zahiv,zika,zionistamerican,zislearn,zmx06,zola1611,zombi
0,-0.200000,0.700000,6.500000,0,6.0,18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

## Vectorize whole dataset for cross-validation

In [36]:
tfidf_vect_cv = TfidfVectorizer(analyzer=clean_text)

X_tfidf_cv = tfidf_vect_cv.fit_transform(data['body_text'])

X_features_cv = pd.concat([data['Sentiment'].reset_index(drop=True), data[cols].reset_index(drop=True), 
                           pd.DataFrame(X_tfidf_cv.toarray(), 
                           columns=tfidf_vect_cv.get_feature_names())], axis=1)

print(X_features_cv.shape)
X_features_cv.head()

(746, 4540)


,Sentiment,Sentiment,Subjectivity,avg_word,numerics,punct%,word_count,,0,04,041,1,10,100,1000,100000,10downingstreet,11k,12,1200pm,12th,14th,14yr,15,17,17yearold,1979,1jasonbuchholz,1k,1st,1u,2,20,2003,201,2016,2017,2018,20yearold,21st,22americanfirst,24,25,27,2face,2xlp,3,30,30k,4,40calbulleit,430,44,448,448psychosi,4way,5,50,518,52,52kiii,53,537,54,5thape,6,60,68yr,6yearold,7,71,720p,7854479161,7pm,8,80,86,89,9,90,911,915fm,99,aaaaafffff,aaronbbrown,aavak,ab,abandon,abcnew,abduct,aberr,abil,abl,ableism,abolish,abondon,abort,abram,absolut,absurd,abt,abus,academ,accept,access,accesss,accid,accident,accompani,accord,account,accur,accus,achiev,acknowledg,acquiesc,across,act,action,activ,actor,actorssing,actual,acut,ad,adapt,adavenh,add,addadhd,addict,addit,addon,address,adhd,adjust,admit,adolesc,adult,advanc,advertis,af,affair,affect,affection,afraid,african,afrin,agap,age,agenda,agent,aggrand,aggressor,agit,ago,agohaven,agre,ahead,aid,aileen,aim,aint,airport,ajdooh,ak47,aka,akikomhart,al,alarm,album,alcohol,alex,alfiesright,alight,aliv,allah,allegedli,allenfrancesmd,allevi,alley,allmus,allow,almost,alon,along,along3,alongsid,alot,alreadi,also,altern,although,alway,alyssamilano,alzheim,alzheimersdiseas,amalgam,amatur,amaz,ambassador,ambit,america,american,americanhotlip,ami,among,amp,amphetamin,amput,amvetsupport,amwrit,analysi,anarchist,anastasiaklynch,anc,andr,andrew,ang,angel,angri,anim,annual,anoth,answer,anthoni,anthonyhopkin,anti,antidepress,antidepressanthttpstco3smhc0hryv,antipsychot,antisemit,antisoci,antonioguterr,anxieti,anymor,anyon,anyth,anyway,anywaya,anywher,ap,apa,apaam18,apollo,appar,apparatchik,appbipolardisord,appear,appli,appoint,...,untreat,updat,upload,upon,upscal,upthegrov,uptight,ur,urbanex,urgent,urself,us,usa,usatoday,use,usedup,useless,user,usual,utopia,utter,utterli,vaccin,vain,valid,valiumtardivedyskinesia,valu,vancouv,variant,variat,variou,vast,vehicl,veil,vengaadash,venom,vent,verbal,veronicatalbot9,versail,version,veteranauthor,via,vibe,vickyjane64,victim,victimhood,victimolog,victoriaj1004,vid,video,videoart,view,villag,villain,villamarshmello,vimeo,vincegottalotta,vinni,vinyl,violenc,violent,violet,viral,virgo,viru,virus,visit,vital,vocal,voguemagazin,voic,volatil,volum,vote,voter,voxdotcom,vs,vu,vulner,w,wa,wafer,waffl,wage,wait,walk,walter,wane,want,war,ward,warlord,warrenelli,warrior,washington,wasnt,wasp,watch,water,wattpad,wave,wax,way,waynedupreeshow,wbuckley59,wcwinc,weak,weapon,wear,weaselzipp,webinar,webmd,websit,wed,wednesdaywisdom,weed,week,weekend,weekli,weep,weight,weird,weirdo,welcom,well,wellb,welldon,wellmindmd,welsh,went,weprogress,west,western,westminst,westsidejack37,what,whatev,whe,whenev,wherea,whether,whine,whip,whisper,white,whiter,whitewolf2226,whoever,whole,whom,whore,whose,wht,wickedsob,wide,widespread,wife,wild,will,williamhaguemp,win,windrush,wing,winner,winnipeg,wipe,wire,withdraw,withdrawn,within,without,wive,wolf,woman,women,wonder,wont,wood,woooooooo,word,work,workshop,world,worri,wors,worship,worst,worth,would,wouldnt,wow,wrap,write,writer,written,wrong,wrote,wrt,wterrorist,wtf,wth,wwwreserveinfus,x,x1mmejor,xavier,xd,xenophob,xi,xijinp,xpress,xx,xxrealist,yak,yall,yashar,ye,yeah,year,yearli,yee,yell,yess,yesterday,yet,yg,yo,yolken,yondu,york,youd,young,your,youth,youtub,youv,yr,yu,yup,z,zahiv,zika,zionistamerican,zislearn,zmx06,zodluc,zoetic,zoetico,zola1611,zombi
0,-0.050000,-0.050000,0.950000,6.250000,0,9.9,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

# 

To evaluate each model, we will use three evaluation metrics:

(1) Accuracy = # predicted correctly / total # of observations

(2) Precision =  # predicted as 1 that are actually 1 (i.e. true positives) /  total # that the model predicted as 1.

(3) Recall = # predicted to be 1 that are actually 1 (i.e. true positives; the same numerator as precision)  /  total # that are                                                                                                                 actually 1                                                                                                                       (instead of the                                                                                                                  total number                                                                                                                    that are predicted as 1)

In [37]:
pd.options.display.float_format = '{:,.2f}'.format
np.set_printoptions(precision=2)

In [38]:
def conFusion_metricsOutput(var_y_test, var_y_pred, modname):
    
    res = []
    
    confusion = metrics.confusion_matrix(var_y_test, var_y_pred)
    TP = confusion[1, 1] #True Positives (TP): we correctly predicted that tweets do have stigma
    TN = confusion[0, 0] #True Negatives (TN): we correctly predicted that tweets don't have stigma
    FP = confusion[0, 1] #False Positives (FP): we incorrectly predicted that tweets do have stigma (a "Type I error")
    FN = confusion[1, 0] #False Negatives (FN): we incorrectly predicted that tweets don't have stigma (a "Type II error")
    
    print(confusion)
    
    ## Classification Accuracy: Overall, how often is the classifier correct?
    accuracy = ((TP + TN) / float(TP + TN + FP + FN))
    #print(metrics.accuracy_score(y_test, y_pred))

    #Classification Error: Overall, how often is the classifier incorrect?
    #Also known as "Misclassification Rate"
    misclassication_rate = ((FP + FN) / float(TP + TN + FP + FN))
    #print(1 - metrics.accuracy_score(y_test, y_pred))

    #Recall/Sensitivity: When the actual value is positive, how often is the prediction correct?
    #How "sensitive" is the classifier to detecting positive instances?
    #Also known as "True Positive Rate"
    recall_tpr = (TP / float(TP + FN))
    #print(metrics.recall_score(y_test, y_pred))

    #Specificity: When the actual value is negative, how often is the prediction correct?
    #How "specific" (or "selective") is the classifier in predicting positive instances?
    specificity = (TN / float(TN + FP))

    #False Positive Rate: When the actual value is negative, how often is the prediction incorrect?
    fpr = (FP / float(TN + FP))

    #Precision: When a positive value is predicted, how often is the prediction correct?
    #How "precise" is the classifier when predicting positive instances?
    precision = (TP / float(TP + FP))
    #print(metrics.precision_score(y_test, y_pred))
    
    
    #false negatives (FN): computer predicted no, but tweet is stigmatising. (Also known as a "Type II error.")
    fnr = (FN / float(TP + FN))
    
    
    res.append([accuracy, misclassication_rate, recall_tpr, specificity, fpr, precision, fnr, FN, FP])
    
    df=pd.DataFrame(res ,columns=['accuracy','misclassication_rate', 'recall_tpr' , 'specificity' , 'fpr', 'precision', 'fnr' , 'FN', 'FP'], index=[modname]).T
    
    return df

In [39]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
import time
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import roc_auc_score

In [40]:
def makeHist(var):
    ax = plt.axes(facecolor ='#E6E6E6')
    ax.set_axisbelow(True)

    # draw solid white grid lines
    plt.grid(color='w', linestyle='solid')

    # hide axis spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    # hide top and right ticks
    ax.xaxis.tick_bottom()
    ax.yaxis.tick_left()

    # lighten ticks and labels
    ax.tick_params(colors='gray', direction='out')
    for tick in ax.get_xticklabels():
        tick.set_color('gray')
    for tick in ax.get_yticklabels():
        tick.set_color('gray')

    # control face and edge color of histogram
    ax.hist(var, edgecolor='#E6E6E6', color='#EE6666');
    plt.xlim(0, 1);
    plt.title('Histogram of predicted probabilities');
    plt.xlabel('Predicted probability of stigma');
    plt.ylabel('Frequency');

In [41]:
def makeROC(var_y_test, var_y_pred):
    # To see the relationship between sensitivity and specificity - plot ROC and calculate AUC
    # ROC curve can help to choose a threshold that balances sensitivity and specificity in a way that makes sense for our particular context
    # IMPORTANT: first argument is true values, second argument is predicted probabilities

    ax = plt.axes(facecolor ='#E6E6E6')
    ax.set_axisbelow(True)

    # draw solid white grid lines
    plt.grid(color='w', linestyle='solid')

    # hide axis spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    # hide top and right ticks
    ax.xaxis.tick_bottom()
    ax.yaxis.tick_left()

    # lighten ticks and labels
    ax.tick_params(colors='gray', direction='out')
    for tick in ax.get_xticklabels():
        tick.set_color('gray')
    for tick in ax.get_yticklabels():
        tick.set_color('gray')

    fpr, tpr, thresholds = metrics.roc_curve(var_y_test, var_y_pred)

    plt.plot(fpr, tpr)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.title('ROC curve for stigma classifier')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.grid(True)

### (1) Random Forest Classifier

#### Bootstrapping/Cross validation 

In [42]:
# Using K Folds as above, to see the metrics at each fold that made up the average above.
# this run is not dropping the insig features

rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1, random_state=0)
k_fold = KFold(n_splits=5)
%time accuracy =        cross_val_score(rf, X_features_cv, data['label'], cv = k_fold, n_jobs = -1, scoring='accuracy')
%time precision_score =  cross_val_score(rf,  X_features_cv, data['label'], cv = k_fold, n_jobs = -1, scoring='precision_weighted')
%time recall_score =    cross_val_score(rf,  X_features_cv, data['label'], cv = k_fold, n_jobs = -1, scoring='recall_weighted')


print(accuracy, precision_score, recall_score)
#Documentation for scoring parameter
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

Wall time: 5.85 s
Wall time: 4.5 s
Wall time: 2.69 s
[0.77 0.81 0.87 0.87 0.83] [0.8  0.82 0.87 0.87 0.83] [0.77 0.81 0.87 0.87 0.83]


In [43]:
# Using K Folds on the test/train split dataset? nope!

#rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1, random_state=0)
#k_fold = KFold(n_splits=5)
#%time cross_val_score(rf, X_train_vect, y_train, cv = k_fold, n_jobs = -1, scoring='accuracy')

#### Grid search to find best parameters for RF models using Train/Test Split data

In [44]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1, random_state=0)
    rf_model = rf.fit(X_train_vect, y_train)
    y_pred = rf_model.predict(X_test_vect)
    precision, recall, fscore, train_support = score(y_test, y_pred, pos_label=1, average="binary")
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    n_est, depth, precision.mean(), recall.mean(),
    (y_pred==y_test).sum() / len(y_pred), 3))

In [45]:
%precision %.2f
np.set_printoptions(precision=2)

for n_est in [10, 50, 100, 150, 200]:
    for depth in [10, 20, 30, 50, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 0.9210526315789473 / Recall: 0.5833333333333334 / Accuracy: 0.8133333333333334
Est: 10 / Depth: 20 ---- Precision: 0.8541666666666666 / Recall: 0.6833333333333333 / Accuracy: 0.8266666666666667
Est: 10 / Depth: 30 ---- Precision: 0.8490566037735849 / Recall: 0.75 / Accuracy: 0.8466666666666667
Est: 10 / Depth: 50 ---- Precision: 0.9361702127659575 / Recall: 0.7333333333333333 / Accuracy: 0.8733333333333333
Est: 10 / Depth: None ---- Precision: 0.8571428571428571 / Recall: 0.7 / Accuracy: 0.8333333333333334
Est: 50 / Depth: 10 ---- Precision: 0.8636363636363636 / Recall: 0.6333333333333333 / Accuracy: 0.8133333333333334
Est: 50 / Depth: 20 ---- Precision: 0.8363636363636363 / Recall: 0.7666666666666667 / Accuracy: 0.8466666666666667
Est: 50 / Depth: 30 ---- Precision: 0.8305084745762712 / Recall: 0.8166666666666667 / Accuracy: 0.86
Est: 50 / Depth: 50 ---- Precision: 0.8596491228070176 / Recall: 0.8166666666666667 / Accuracy: 0.8733333333333333
Est: 5

We see these parameters produce best results:

Est: 50 / Depth: 50 ---- Precision: 0.8909090909090909 / Recall: 0.8166666666666667 / Accuracy: 0.8866666666666667


Est: 50 / Depth: None ---- Precision: 0.9056603773584906 / Recall: 0.8 / Accuracy: 0.8866666666666667

#### RF Model with Train/Test Split parameters 

##### Dropping insignification features 

In [46]:
#rf hold out test dropping insig features, using hyperparameters based on our grid search above

rf = RandomForestClassifier(n_estimators=50, max_depth=50, n_jobs=-1, random_state=0) 
# n_estimators  is how many decision trees that will be built within your random forest, so the default is 10. These defaults mean, your random forest would build 10 decision trees of unlimited depth. Then, there would be a vote among these 10 trees to determine the final prediction.
# max depth basically means that it will build each decision tree until it minimizes some loss criteria.

start = time.time()
rf_model = rf.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = rf_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label=1, average="binary")

print('Fit time: {} / Predict time: {} ----\n\n **Precision: {} \n\n **Recall: {} \n\n **Accuracy: {}'.format(
    fit_time, pred_time, precision.mean(), recall.mean(), (y_pred==y_test).sum()/len(y_pred)))

Fit time: 0.26002025604248047 / Predict time: 0.053003787994384766 ----

 **Precision: 0.8596491228070176 

 **Recall: 0.8166666666666667 

 **Accuracy: 0.8733333333333333


In [47]:
res_df = conFusion_metricsOutput(y_test, y_pred, modname = 'rf_model')
res_df

[[82  8]
 [11 49]]


,rf_model
accuracy,0.87
misclassication_rate,0.13
recall_tpr,0.82
specificity,0.91
fpr,0.09
precision,0.86
fnr,0.18
FN,11.00
FP,8.00


In [48]:
stigmacount = 0
for i in y_pred:
    if i == 1:
        stigmacount += 1
print(stigmacount)
percentage_stigma = (stigmacount/150)* 100
percentage_nonstigma = ((150-stigmacount)/150)*100
len(y_pred)

57


150

In [49]:
print('Stigma: ' + str(percentage_stigma) + ' No Stigma: '+ str(percentage_nonstigma))

Stigma: 38.0 No Stigma: 62.0


### (4) Support Vector Machine (SVM) Classifier 

In [50]:
from sklearn.svm import SVC

### (5) SVM with a linear kernal 

In [51]:
svmClas = SVC(kernel='linear', probability=True)
svmClas.fit(X_train_vect, y_train)
pred = svmClas.predict(X_test_vect)

In [67]:
pred

array([0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0])

In [52]:
res_df = res_df.join(conFusion_metricsOutput(y_test, pred, 'svmLinear'))
res_df

[[80 10]
 [ 3 57]]


,rf_model,svmLinear
accuracy,0.87,0.91
misclassication_rate,0.13,0.09
recall_tpr,0.82,0.95
specificity,0.91,0.89
fpr,0.09,0.11
precision,0.86,0.85
fnr,0.18,0.05
FN,11.00,3.00
FP,8.00,10.00


In [53]:
stigmacount = 0
for i in pred:
    if i == 1:
        stigmacount += 1
print(stigmacount)
percentage_stigma = (stigmacount/150)* 100
percentage_nonstigma = ((150-stigmacount)/150)*100
len(pred)

67


150

In [54]:
print('Stigma: ' + str(percentage_stigma) + ' No Stigma: '+ str(percentage_nonstigma))

Stigma: 44.666666666666664 No Stigma: 55.333333333333336


In [55]:
# Below was done for Batch1 and Batch2
df_forPred = pd.read_excel('Batch2.xlsx', names = ['body_text'])

In [56]:
df_forPred

,body_text
0,Schizophrenia is Pellagra — Abram Hoffer Quote...
1,Winter that psychotic girlfriend that won't le...
2,lmao https://t.co/awZSSyLITI
3,Donald Trump reveals his plan to fight back ag...
4,@ideoautiste Schizophrenia
...,...
477,Your #TrumpDerangementSyndrome has advanced to...
478,Schizophrenia Simulation. Was listening to a p...
479,"@rainnwilson ""Curious George Develops Drug-Ind..."
480,"#BellLetsTalk about depression and anxiety, wi..."


In [57]:
df_forPred = sentAnal(df_forPred)
df_forPred['punct%'] = df_forPred['body_text'].apply(lambda x: count_punct(x))


# Average Word Length. simply take the sum of the length of all the words and divide it by the total length of the tweet as defined in function above
df_forPred['avg_word'] = df_forPred['body_text'].apply(lambda x: avg_word(x))

# Number of Words in tweet
df_forPred['word_count'] = df_forPred['body_text'].apply(lambda x: len(str(x).split(" ")))





# number of numerics in tweet
df_forPred['numerics'] = df_forPred['body_text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))



In [58]:
tfidf_test_pred = tfidf_vect_fit.transform(df_forPred['body_text'])


X_test_vect_pred = pd.concat([df_forPred[cols].reset_index(drop=True), 
           pd.DataFrame(tfidf_test_pred.toarray(), columns=tfidf_vect.get_feature_names())], axis=1)

X_test_vect_pred.shape

(482, 3836)

In [59]:
X_test_vect_pred

,Sentiment,Subjectivity,avg_word,numerics,punct%,word_count,,0,041,1,10,100,1000,100000,12,1200pm,12th,14th,14yr,15,17yearold,1979,1jasonbuchholz,1k,1st,1u,2,20,2003,2016,2017,2018,20yearold,22americanfirst,24,25,27,2xlp,3,30,30k,4,40calbulleit,448,448psychosi,4way,5,50,518,52kiii,53,537,6,60,68yr,6yearold,7,71,720p,7854479161,7pm,8,80,86,89,9,90,911,915fm,99,aaaaafffff,aavak,ab,abandon,abcnew,abduct,aberr,abil,abl,ableism,abolish,abondon,abort,abram,absolut,absurd,abt,abus,academ,accept,access,accesss,accid,accident,accompani,accord,account,accur,accus,achiev,acknowledg,acquiesc,across,act,action,activ,actorssing,actual,acut,ad,adapt,adavenh,add,addadhd,addict,addit,addon,address,adhd,adjust,adolesc,adult,advanc,advertis,affair,affect,afraid,african,afrin,agap,age,agenda,agent,aggrand,aggressor,agit,ago,agohaven,agre,ahead,aid,aileen,aim,airport,aka,al,album,alcohol,alex,aliv,allah,allegedli,allenfrancesmd,allevi,alley,allmus,allow,alon,along,along3,alongsid,alot,alreadi,also,altern,although,alway,alyssamilano,alzheim,alzheimersdiseas,amatur,amaz,america,american,americanhotlip,ami,among,amp,amphetamin,amput,amvetsupport,amwrit,analysi,anarchist,anc,andr,andrew,ang,angel,angri,anim,annual,anoth,answer,anti,antidepress,antidepressanthttpstco3smhc0hryv,antipsychot,antisemit,antisoci,anxieti,anymor,anyon,anyth,anyway,anywaya,anywher,ap,apa,apollo,appar,appbipolardisord,appear,appli,appointe,approach,appropri,april,aqa,aqapsycholog,arab,arbit,area,arent,arizona,arizonakayt,armi,arnold,aroesti,around,arriv,arsen,art,artefr,arthriti,articl,asf,asham,ashkenazi,ashwini,ashwininangia,asia,ask,aspd,aspir,ass,assassin,assault,assess,asshol,...,tutor,tv,tvbandnew,tweet,twilightavian,twist,twitter,two,typic,typpex,typpexprog,tyrant,u,u2,ualberta,ubc,uk,ukchang,um,umehoma,umma,ummah,ummalmumineen,un,unbridl,uncontrol,understand,underukslaw,undiagnos,unfair,unfold,unfollowblock,uniform,uninform,union,uniqu,unit,unitedfronthk,univers,unjustifi,unlac,unless,unlik,unmed,unpick,unquest,unreal,unrel,unsaf,unse,unstabl,untreat,updat,upload,upon,upscal,upthegrov,ur,urbanex,urgent,urself,us,usa,usatoday,use,usedup,useless,user,usual,utopia,utter,utterli,vain,valid,valiumtardivedyskinesia,valu,vancouv,variant,variat,variou,vast,vehicl,vengaadash,venom,vent,verbal,versail,version,via,vibe,vickyjane64,victim,victimolog,victoriaj1004,vid,video,view,villag,villain,villamarshmello,vincegottalotta,vinni,vinyl,violenc,violent,violet,viral,virgo,viru,visit,vital,voguemagazin,voic,volatil,volum,vote,vs,vu,w,wa,wafer,waffl,wage,wait,walk,walter,want,war,ward,warlord,warrenelli,washington,wasnt,wasp,watch,water,wave,way,wbuckley59,wcwinc,weak,weapon,weaselzipp,webinar,webmd,websit,wed,wednesdaywisdom,weed,week,weekend,weight,weird,weirdo,welcom,well,welldon,wellmindmd,welsh,went,weprogress,west,western,westminst,westsidejack37,what,whatev,whe,whenev,wherea,whether,whip,white,whiter,whole,whom,whore,wht,wide,widespread,wife,will,williamhaguemp,win,wing,winnipeg,wipe,wire,withdraw,withdrawn,within,without,woman,women,wonder,wood,woooooooo,word,work,world,worri,wors,worship,worst,worth,would,wouldnt,wow,wrap,write,writer,written,wrong,wrt,wterrorist,wtf,wth,wwwreserveinfus,xavier,xd,xi,xijinp,xpress,xxrealist,yall,yashar,yeah,year,yearli,yess,yesterday,yet,yg,yo,yolken,yondu,york,young,your,youtub,youv,yr,yu,zahiv,zika,zionistamerican,zislearn,zmx06,zola1611,zombi
0,0.00,0.00,7.88,0,7.90,8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00

In [60]:
y_pred_rf = rf.predict(np.nan_to_num(X_test_vect_pred))

In [61]:
y_pred_svm = svmClas.predict(np.nan_to_num(X_test_vect_pred))

In [62]:
texts = [i for i in df_forPred['body_text']]

In [63]:
Dicty = {}
Dicty['Tweet'] = texts
Dicty['Random Forest'] = y_pred_rf
Dicty['SVM' ] = y_pred_svm

In [64]:
final_df = pd.DataFrame.from_dict(Dicty)

In [65]:
final_df

,Tweet,Random Forest,SVM
0,Schizophrenia is Pellagra — Abram Hoffer Quote...,0,0
1,Winter that psychotic girlfriend that won't le...,1,1
2,lmao https://t.co/awZSSyLITI,0,0
3,Donald Trump reveals his plan to fight back ag...,1,1
4,@ideoautiste Schizophrenia,0,0
...,...,...,...
477,Your #TrumpDerangementSyndrome has advanced to...,0,0
478,Schizophrenia Simulation. Was listening to a p...,0,0
479,"@rainnwilson ""Curious George Develops Drug-Ind...",0,0
480,"#BellLetsTalk about depression and anxiety, wi...",0,0


In [66]:
final_df.to_csv('')